<a href="https://colab.research.google.com/github/Kingzel/pyMD/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
from pymdroutines import gen_multichoice_features , gen_binary_features , gen_singlechoice_features

Read Evidences/Symptoms

In [7]:
evids = pd.read_json("data\\release_evidences.json")

From the evidences, prepare to explode the predictors for binary categorical, multi choice categroical and single choice categorical evidences accoridngly to be interger encoded (one hot encoding + interger ordinal encoding where neccessary)

In [9]:
m_collection = gen_multichoice_features(evids,'S')
c_collection =gen_singlechoice_features(evids,'S')
b_collection = gen_binary_features(evids)

Manually reincluding SEX and AGE as predictors.

In [10]:
all_collections = b_collection+m_collection+c_collection
all_collections.extend(['AGE','SEX'])
print(len(all_collections))



914


Import training dataset batches and slice into (predictors and target)

In [ ]:
from pymdroutines import parse_data
parsed = parse_data(["output\\output_1.csv","output\\output_2.csv","output\\output_3.csv","output\\output_4.csv","output\\output_5.csv"])
train_patients,y = pd.concat(parsed[0::2], ignore_index=True),pd.concat(parsed[1::2],ignore_index=True)

Creating a dummy feature space with predictors as accumulated in `all_collections` and populating with `len(train_patients)` number of rows filled with 0s. 

In [11]:
from pymdroutines import gen_empty
new_df = gen_empty(all_collections,len(train_patients)) 

#Verifying shape of the dummy feature space and the values of the first few rows and columns.
print(new_df.shape,"\n")
print(new_df.iloc[0:2,:4])

NameError: name 'train_patients' is not defined

Actually populating the feature space with patient data (symptoms, sex and age are loaded in structural accordance.)

In [ ]:
from pymdroutines import populate
new_df = populate(new_df,train_patients)

In [ ]:
#Verifying shape of the dummy feature space and the values of the first few rows and columns.
print(new_df.shape)
print(new_df.iloc[0:2,:4])

Splitting the dataset into training and testing sets.

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(new_df, y, test_size = 0.35)

Fitting a Random Forest Classifier on the training set (10 trees)

In [ ]:
clf = RandomForestClassifier(n_estimators=25, warm_start=True,max_depth=15)
clf.fit(X_train, y_train)
print(len(X_train))


Predicting and evlauating classification accuracy of the RF model.

In [ ]:
y_pred  = clf.predict(X_test)
correct = 0
total =0
for predicted, actual in zip(y_pred,y_test):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,'\n',correct/total,'accuracy')

Saving the trained model using pickle binary dumps.

In [ ]:
import pickle as p

with open('trained_model.bin', 'wb') as f:
    p.dump(clf, f)
    p.dump(all_collections,f)


<h1>TESTING AND DEPLOYMENT</H1>

In [12]:
import pickle as p
with open('trained_model.bin', 'rb') as f:
    rf =  p.load(f)
    cols  = p.load(f)
print(len(cols))

914


In [13]:
from pymdroutines import parse_data , populate, gen_empty
parsed = parse_data(["data\\release_test_patients\\release_test_patients.csv"])
test_patients = parsed[0]
y=parsed[1]
new_df = gen_empty(cols,len(test_patients))


In [14]:
new_df = populate(new_df,test_patients)

KeyboardInterrupt: 

In [ ]:
y_pred  = rf.predict(new_df)
correct = 0
total =0
for predicted, actual in zip(y_pred,y):
  if predicted == actual:
    correct+=1
  total+=1
print(correct,'out of',total,correct/total,'accuracy')
# print(new_df.head())

In [ ]:

import visualize
import importlib
importlib.reload(visualize)
num_of_trees = len(rf.estimators_)
for i in range(2):
    visualize.tvisual(10,cols,rf.estimators_[i])

In [26]:
import operator
dummy = gen_empty(cols,1)


dummy['E_204_@_V_0']=1
dummy['E_204_@_V_1']=1
dummy['E_178']=1
dummy['E_91']=1
dummy['E_89']=1
dummy['E_89']=1
dummy['E_73']=1

dummy['AGE'] = 60

['E_51', 'E_66', 'E_73', 'E_97', 'E_148', 'E_201', 'E_204_@_V_1']
y_probabs = rf.predict_proba(dummy)
y_predict = rf.predict(dummy)
zipped = zip(rf.classes_,y_probabs.tolist()[0])
prob_to_label ={}
for disease,probab in zipped:
    prob_to_label[disease] = probab
prob_to_label = dict( sorted(prob_to_label.items(), key=operator.itemgetter(1), reverse=True))

print(prob_to_label,y_predict,sep = '\n'*2)



{'Larygospasm': 0.07625649952229328, 'Tuberculosis': 0.06783303065712293, 'Bronchiectasis': 0.06506527857581948, 'Acute COPD exacerbation / infection': 0.06342029325470247, 'Atrial fibrillation': 0.05824130743344675, 'Guillain-Barré syndrome': 0.05681329114502012, 'Scombroid food poisoning': 0.04928480371978531, 'Allergic sinusitis': 0.04679632272947479, 'Bronchospasm / acute asthma exacerbation': 0.04613066153934982, 'Ebola': 0.044068981856963604, 'Acute dystonic reactions': 0.042848680608711665, 'Myasthenia gravis': 0.04086125567897938, 'Whooping cough': 0.03934581922528195, 'PSVT': 0.026165237004971288, 'Unstable angina': 0.017787034377572036, 'Croup': 0.016256239719810633, 'Stable angina': 0.016013316539635235, 'Pericarditis': 0.015724977591475862, 'GERD': 0.01563615380640672, 'Possible NSTEMI / STEMI': 0.014803290607548281, 'Pulmonary neoplasm': 0.014617884809136188, 'Boerhaave': 0.01359520036993759, 'Bronchitis': 0.010218433987330276, 'Acute pulmonary edema': 0.010111230183723105

DONE